# Analysis of Chicago Police Homicide Clearance Data


In [1]:
import re
import pandas as pd
from src import schemas


## Data import and cleaning


In [2]:
rename_columns = lambda df: df.rename(
    columns=lambda col: re.sub(
        r"\s+", "_", re.sub(r"[^A-z]+", " ", col.strip()).lower().strip()
    )
)

status = (
    pd.read_excel(
        "input/18731-P744333_Homicide_status_2010-YTD.xlsx",
        sheet_name=1,
        parse_dates=["Injury Date", "Death Date", "CompStat Date", "Date Cleared"],
    )
    .pipe(rename_columns)
    .replace(
        {
            "cleared_i": {"Y": True, "N": False},
            "victim_sex": {"M": "male", "F": "female", "X": "unknown/other"},
        }
    )
    .rename(columns={"homicide": "homicide_no", "rd": "case_no"})
    .assign(
        incident_year=lambda df: df.compstat_date.dt.year,
        clearance_year=lambda df: df.date_cleared.dt.year,
    )
    .drop(["victim_sex", "victim_age"], axis=1)
    .drop_duplicates()
    .pipe(schemas.status_schema)
)

victims = (
    pd.read_csv(
        "input/Violence_Reduction_-_Victims_of_Homicides_and_Non-Fatal_Shootings.csv",
        parse_dates=["DATE"],
    )
    .pipe(rename_columns)
    .query("victimization_primary == 'HOMICIDE'")
    .rename(columns={"race": "victim_race", "case_number": "case_no"})[
        ["case_no", "victim_race"]
    ]
    .replace(
        {
            "victim_race": {
                "BLK": "black",
                "WHI": "white",
                "WWH": "hispanic",
                "WBH": "hispanic",
                "API": "api",
                "I": "indian",
                "UNKNOWN": "unknown",
            }
        }
    )
    .pipe(schemas.victims_schema)
)


## Analysis


The number of homicides nearly doubled over about a decade. In 2010, there were 520, and in 2020, there were 842.


In [3]:
total_incidents = (
    status.groupby("incident_year")
    .homicide_no.nunique()
    .to_frame("total_incidents")
    .query("incident_year >= 2010")
)
total_incidents


,total_incidents
incident_year,
2010,441
2011,446
2012,510
2013,425
2014,424
2015,493
2016,780
2017,660
2018,579


Chicago PD's overall clearance rate appears to be relatively good for the city with the most homicides of any in the country -- 46% in 2020, which is just a few points below the national average.


In [4]:
overall_clearance = (
    total_incidents.join(
        status.query("cleared_i == True")
        .groupby("clearance_year")
        .case_no.nunique()
        .to_frame("total_clearances")
    )
    .assign(clearance_rate=lambda df: df.total_clearances / df.total_incidents)
    .query("incident_year >= 2010")
)

overall_clearance


,total_incidents,total_clearances,clearance_rate
incident_year,,,
2010.0,441,206,0.467120
2011.0,446,201,0.450673
2012.0,510,178,0.349020
2013.0,425,224,0.527059
2014.0,424,210,0.495283
2015.0,493,210,0.425963
2016.0,780,207,0.265385
2017.0,660,212,0.321212
2018.0,579,247,0.426598


But when you remove exceptional clearances, the picture is much worse. In 2020, only about a quarter of homicides were cleared by arrest

In [5]:
arrest = total_incidents.join(
    status.query("(cleared_i == True) & (cleared_exceptionally_by.isna())")
    .groupby("clearance_year")
    .case_no.nunique()
    .to_frame("total_clearances")
).assign(clearance_rate=lambda df: df.total_clearances / df.total_incidents)

arrest


,total_incidents,total_clearances,clearance_rate
incident_year,,,
2010.0,441,133,0.301587
2011.0,446,155,0.347534
2012.0,510,139,0.272549
2013.0,425,161,0.378824
2014.0,424,171,0.403302
2015.0,493,143,0.290061
2016.0,780,170,0.217949
2017.0,660,134,0.203030
2018.0,579,140,0.241796


The share of clearances that were made by arrest is also decreasing. In 2013, more than 80% of clearances were by arrest, while in 2020 it was less than 50%.

In [6]:
def add_multiindex_level(df, colname):
    df.columns = pd.MultiIndex.from_tuples([(colname, c) for c in df.columns])
    return df


merge_cols = ["incident_year", "total_incidents"]

compare_df = (
    overall_clearance.reset_index()
    .set_index(merge_cols)
    .pipe(add_multiindex_level, "all clearances")
    .merge(
        arrest.reset_index()
        .set_index(merge_cols)
        .pipe(add_multiindex_level, "arrest only"),
        on=merge_cols,
    )
    .assign(
        pct_clearances_by_arrest=lambda df: df[("arrest only", "total_clearances")]
        / df[("all clearances", "total_clearances")]
    )
)

compare_df


all clearances                 \
                              total_clearances clearance_rate   
incident_year total_incidents                                   
2010.0        441                          206       0.467120   
2011.0        446                          201       0.450673   
2012.0        510                          178       0.349020   
2013.0        425                          224       0.527059   
2014.0        424                          210       0.495283   
2015.0        493                          210       0.425963   
2016.0        780                          207       0.265385   
2017.0        660                          212       0.321212   
2018.0        579                          247       0.426598   
2019.0        500                          253       0.506000   
2020.0        774                          339       0.437984   
2021.0        800                          383       0.478750   
2022.0        174                          102       0.586207   

                                   arrest only                 \
                              total_clearances clearance_rate   
incident_year total_incidents                                   
2010.0        441                          133       0.301587   
2011.0        446                          155       0.347534   
2012.0        510                          139       0.272549   
2013.0        425                          161       0.378824   
2014.0        424                          171       0.403302   
2015.0        493                          143       0.290061   
2016.0        780                          170       0.217949   
2017.0        660                          134       0.203030   
2018.0        579                          140       0.241796   
2019.0        500                          109       0.218000   
2020.0        774                          198       0.255814   
2021.0        800                          189       0.236250   
2022.0        174                           65       0.373563   

                              pct_clearances_by_arrest  
                                                        
incident_year total_incidents                           
2010.0        441                             0.645631  
2011.0        446                             0.771144  
2012.0        510                             0.780899  
2013.0        425                             0.718750  
2014.0        424                             0.814286  
2015.0        493                             0.680952  
2016.0        780                             0.821256  
2017.0        660                             0.632075  
2018.0        579                             0.566802  
2019.0        500                             0.430830  
2020.0        774                             0.584071  
2021.0        800                             0.493473  
2022.0        174                             0.637255

At the same time, the share of clearances marked "bar to prosecute" is increasing rapidly. In 2016, it was less than 10 percent. In 2021, it was 34.

In [7]:
# exceptional clearance comparison
ec_compare = (
    status.query("cleared_i == True")
    .assign(
        cleared_exceptionally_by=(
            lambda df: df.cleared_exceptionally_by.fillna("arrest").str.lower()
        )
    )
    .rename(columns={"cleared_exceptionally_by": "clearance_description"})
    .pipe(
        lambda df: pd.crosstab(
            index=df.clearance_year,
            columns=df.clearance_description,
            values=df.case_no,
            aggfunc="nunique",
        )
    )
    .pipe(lambda df: df.div(df.sum(axis=1), axis=0))
)

ec_compare


clearance_description,arrest,bar to prosecute,death of offender
clearance_year,,,
2010.0,0.645631,0.252427,0.101942
2011.0,0.771144,0.174129,0.054726
2012.0,0.780899,0.123596,0.095506
2013.0,0.718750,0.218750,0.062500
2014.0,0.814286,0.123810,0.061905
2015.0,0.677725,0.175355,0.146919
2016.0,0.821256,0.106280,0.072464
2017.0,0.632075,0.221698,0.146226
2018.0,0.566802,0.323887,0.109312


In [8]:
with pd.ExcelWriter("output/chicago_police_clearance_rate.xlsx") as writer:
    compare_df.to_excel(writer, sheet_name="Clearance rates")
    ec_compare.to_excel(writer, sheet_name="Share of clearances by type")
